In [1]:
import os
from contextlib import contextmanager

import numpy as np
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from sshtunnel import SSHTunnelForwarder

In [2]:
load_dotenv("misc/.env")

# SSH and DB configuration from environment variables
SSH_HOST = os.getenv("SSH_HOST")
SSH_PORT = int(os.getenv("SSH_PORT", 22))
SSH_USER = os.getenv("SSH_USER")
SSH_PASSWORD = os.getenv("SSH_PASSWORD")

DB_HOST = os.getenv("DB_HOST")
DB_PORT = int(os.getenv("DB_PORT", 5432))
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")


@contextmanager
def __create_ssh_tunnel():

    tunnel = None
    try:

        tunnel = SSHTunnelForwarder(
            (SSH_HOST, SSH_PORT),
            ssh_username=SSH_USER,
            ssh_password=SSH_PASSWORD,
            remote_bind_address=(DB_HOST, DB_PORT),
            local_bind_address=("127.0.0.1", 5432),
        )
        tunnel.start()

        yield tunnel
    except Exception as e:

        raise
    finally:
        if tunnel:
            tunnel.stop()


@contextmanager
def create_db_engine():
    """
    Creates an SQLAlchemy engine connected through the SSH tunnel.
    Yields the engine for use in a with statement.
    """
    with __create_ssh_tunnel() as tunnel:
        engine = None
        try:

            connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{tunnel.local_bind_port}/{DB_NAME}"

            engine = create_engine(
                connection_string, echo=False, pool_pre_ping=True, pool_recycle=3600
            )
            yield engine
        except SQLAlchemyError as e:

            raise
        finally:
            if engine:
                engine.dispose()


def create_crowd_levels(df, target_district):
    target_column = f'{target_district.replace("_", " ")}_c_lvl'

    out = pd.qcut(
        df[target_district].rank(method="first"),
        q=[0, 0.3, 0.7, 1],  # q=[0, 0.2, 0.45, 0.65, 0.8, 1] for 5 bins
        labels=[0, 1, 2],
    )

    return out.astype(np.uint8), target_column

In [4]:
with create_db_engine() as engine:
    with engine.connect() as conn:
        query = f"SELECT * FROM crowdedness"
        chunks = pd.read_sql_query(sql=query, con=conn, chunksize=10**6)
    df = pd.concat(chunks, ignore_index=True, axis=0)

In [5]:
df = (
    df.pivot_table(
        index="timestamp",
        columns="district_id",
        values="crowd",
        aggfunc="sum",
    )
    .ffill()
    .bfill()
    .astype(np.uint16)
    .sort_values(by="timestamp")
    .reset_index()
)

df.set_index("timestamp", inplace=True)

In [23]:
cols_to_keep = [col for col in df.columns if df[col].var() > 0]
filtered_df = df[cols_to_keep]

In [40]:
from sklearn.preprocessing import OrdinalEncoder


target_columns = {}
for target_district in filtered_df.columns:
    print(target_district)
    out, target_column = create_crowd_levels(df, target_district)
    print(np.unique(out))
    # if diff := set([0, 1, 2]) - set(out):
    #     diff = diff.pop()
    #     print(diff)
    #     encoder = OrdinalEncoder(
    #         categories=[[0, 1, 2]],
    #         handle_unknown="use_encoded_value",
    #         unknown_value=diff,
    #     )
    #     out = encoder.fit_transform(out)
    #     print(np.unique(out))
    # target_columns[target_column] = out
# target_labels = list(target_columns.keys())
# df[target_labels] = pd.DataFrame(target_columns)

Bedrijvenpark Noord-West
[0 1 2]
Bedrijventerrein Schieveen
[0 1 2]
Charlois
[0 1 2]
Delfshaven
[0 1 2]
Feijenoord
[0 1 2]
Hillegersberg-Schiebroek
[0 1 2]
Hoek van Holland
[0 1 2]
Hoogvliet
[0 1 2]
Kralingen-Crooswijk
[0 1 2]
Nieuw Mathenesse
[0 1 2]
Noord
[0 1 2]
Overschie
[0 1 2]
Pernis
[0 1 2]
Prins Alexander
[0 1 2]
Rivium
[0 1 2]
Rotterdam Centrum
[0 1 2]
Spaanse Polder
[0 1 2]
Vondelingenplaat
[0 1 2]
Waalhaven
[0 1 2]
Ĳsselmonde
[0 1 2]


In [64]:
from sklearn.model_selection import train_test_split


def split_dataset(lagged_df, n_targets, district, step):
    temp = lagged_df.copy(deep=True)
    temp[district] = temp[district].shift(-step)
    temp = temp.dropna()
    temp[district] = temp[district].astype(np.uint8)

    X = temp[temp.columns[:-n_targets]]
    y = temp[district]

    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.3,
        random_state=42,
        shuffle=False,
    )
    return X_train, X_test, y_train, y_test